In [1]:
import pymysql
import os
import re
import copy
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from lxml import etree
import json
import time

In [2]:
connection = pymysql.connect(host='127.0.0.1',
                             user='data',
                             password='data',
                             db='mag-new-160205',
                             charset='utf8mb4',
                             port=3306,
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

In [3]:
author_name_set = set()
data_folder = 'data'
link_data_file_name = 'links_10000'
link_data = open(os.path.join(data_folder, link_data_file_name+'.txt'), 'r')
for line in link_data:
    aus = line.split('\t')[:2]
    for au in aus:
        author_name_set.add(au.replace('_', ' '))
print (len(author_name_set))

7945


In [ ]:
search_id_str = "SELECT AuthorID FROM Authors WHERE AuthorName = '%s'"
# search_fos_str = "SELECT AuthorFOS.FieldOfStudyIDMappedToKeyword, \
#                                 FieldsOfStudy.FieldsOfStudyName, \
#                                 FieldsOfStudy.FieldsOfStudyLevel \
#                                 FROM AuthorFOS, FieldsOfStudy WHERE \
#                                 AuthorFOS.AuthorID = '%s' and FieldsOfStudy.FieldsOfStudyID = AuthorFOS.FieldOfStudyIDMappedToKeyword"
search_fos_str = "SELECT FieldOfStudyIDMappedToKeyword \
                                FROM AuthorFOS WHERE \
                                AuthorID = '%s' "
res_dict = {}
start = time.time()
for i, au in enumerate(author_name_set):
#     print (search_id_str % au)
    if (i+1)%500 == 0:
        print ('Process: %d, Time: %.3f', (i+1), time.time()-start)
        start = time.time()
    res_dict[au.replace(' ', '_')] = []
    cursor.execute(search_id_str % au)
    id_res = cursor.fetchall()
#     print ('len idres', len(id_res))
    if id_res:
        for id_row in id_res:
#             print ('idrow', id_row)
            cursor.execute(search_fos_str % id_row['AuthorID'])
            fos_res = cursor.fetchall()
#             print ('len fos_res', len(fos_res))
            if fos_res:
                for fos_row in fos_res:
#                     print ('fos_row', fos_row)
                    res_dict[au.replace(' ', '_')].append(fos_row['FieldOfStudyIDMappedToKeyword'])
#                     print (fos_row['FieldOfStudyIDMappedToKeyword'])
#     break


In [5]:
derive_file = open(os.path.join(data_folder, link_data_file_name+'_with_fos.txt'), 'w')
for k,v in res_dict.items():
    derive_file.write(k+'\t')
    for fos in v:
        derive_file.write(fos+' ')
    derive_file.write('\n')